# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Project 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: ** 
O tipo de problema de aprendizagem supervisionada é classificação. Dado uma coleção de estudantes o problema visa identificar aqueles que podem precisar de intervenção antecipada e aqueles que não podem precisar, ou seja, será necessário rotular se determinado estudante sofrerá intervenção ou não, baseado nos dados dos atributos. Logo, a saída do classificador será binária (discreta), ou seja, sim ou não. Mostrando que é um problema de classificação.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [42]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [43]:
# TODO: Calcule o número de estudante
n_students = len(student_data)

# TODO: Calcule o número de atributos
n_features = len(student_data.columns)-1

# TODO: Calcule o número de alunos aprovados
n_passed = student_data.passed.value_counts()['yes']

# TODO: Calcule o número de alunos reprovados
n_failed = student_data.passed.value_counts()['no']

# TODO: Calcule a taxa de graduação
grad_rate = (n_passed*1.0 / n_students)*100.0

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [44]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo, 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
pd.set_option('expand_frame_repr', True)
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (em inglês: _dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [45]:
from sklearn.preprocessing import Normalizer

def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

# ******* Normaliza os dados **********
normer = Normalizer()
X = normer.fit_transform(X_all.values)
X_all = pd.DataFrame(data=X, columns=list(X_all.columns))

print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
print X_all.head()


Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
   school_GP  school_MS     sex_F  sex_M       age  address_R  address_U  \
0   0.046424        0.0  0.046424    0.0  0.835629        0.0   0.046424   
1   0.051434        0.0  0.051434    0.0  0.874386        0.0   0.051434   
2   0.049629        0.0  0.049629    0.0  0.744438        0.0   0.049629   
3   0.055989        

### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [46]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn import cross_validation

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
X_all, y_all, stratify=y_all, train_size=num_train, test_size=num_test, random_state=0)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **
Os três modelos escolhidos são: Gaussian Naive Bayes (GaussianNB), Árvores de Decisão e Máquinas de vetores de suporte (SVM). Os três modelos, além de serem modelos que lidam com problemas de classificação binária, como é o caso, conseguem lidar com um maior número de features (atributos) e, mesmo aplicados em um conjunto de dados pequeno, conseguem obter valores bons valores de acuracidade para esse tipo de classificação.  

1 . GaussianNB

   1.1. Aplicação: detecção de spam de perfis em redes sociais. Esse problema consiste em identificar se perfis em redes sociais são realmente reais baseado nas informaçoes repassadas no perfil e no timeline. NB é usado para resolver esse problema de forma similar como é usado tradicionamente para detectar spam de emails. Fonte: http://theory.stanford.edu/~dfreeman/papers/namespam.pdf
    
   1.2. Vantagens: seu algoritmo é simples, tem ótimo desempenho computacional, principalemte na fase de treino, converge mais rápido por isso exige um tamanho de treino menor. 
    
   1.3. Desvantagens: não pode ser aplicados em conjunto de dados onde os atributos (features) sejam dependentes, como por exemplos uma hierarquia ou uma relação entre atributos, pois o modelo assume como premissa a forte condicional independência entre os atributos. 
    
   1.4. Candidato ao problema: é um bom candidato pois o conjunto de dados é pequeno, seus atributos são independentes e a saída do problema é binária.
    
    
2 . Árvore de Decisão

   2.1. Aplicação: área de finanças. Como por exemplo, em precificação de produtos de acordo com a data de vencimento, levando em consideração atributos como quantidade de estoque, tipo de venda, local da venda, data, preço atual. Fonte: http://www.investopedia.com/articles/financial-theory/11/decisions-trees-finance.asp 
    
   2.2. Vantagens: fácil de interpretar, de explicar e tem boa expressividade. Lidam bem com muitos atributos, sejam discretos ou contínuos, mesmo se os dados não são linearmente separáveis. Apresentam, em média, alta acurácia. 
    
   2.3. Desvantagens: tendem a sobreajustar, principalmente quando há uma grande quantidade de atributos e as árvores geradas são complexas. Não lida bem com reaprendizado ou reforço, ou seja, toda vez que a árvore tiver que aprender com novos exemplos de treino ela terá que ser reconstruída.
    
   2.4. Candidato ao problema: é um bom cadidato pois há muitos atributos e eles são discretos, os dados dos atributos estão limpos (pouco ruído e pouca ausência nos dados), e a saída do problema é binária.
    
    
3 . SVM

   3.1. Aplicação: avaliação de risco de crédito. Análise e avaliação de risco de crédito é uma atividade crucial na rede de bancos publica e privada, pois os bancos precisam discriminar os bons dos maus credores. SVM é amplamente utilizado nessa tarefa de classificação. Fonte: https://www.computer.org/csdl/proceedings/gcis/2009/3571/04/3571d049.pdf
    
   3.2. Vantagens: ideais a serem aplicados em domínios onde há clara margem de separação dos dados que se quer classificar. Capacidade de lidar com conjuntos de dados complexo, com muitas dimensões, como por exemplo, obtém altos valores de acurária quando aplicados para classificação de textos em espaços de alta dimensão. Pouco overfitting em conjunto de dados limpos.  
    
   3.3. Desvantagens: difíceis de interpretar, consomem muita memória, não trabalham bem em grandes conjuntos de dados, pois o tempo computacional do treino é alto. Não funciona bem em conjunto de dados com grande quantidade de ruídos. 
    
   3.4. Candidato ao problema: é um bom candidato pois o conjunto de dados é pequeno, está com pouco ruído, os valores dos atributos são discretos e a saída do modelo é binária.
    

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [47]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    t = end - start
    #print "O modelo foi treinado em {:.4f} segundos".format(t)
    return t

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    t = end - start
    #print "As previsões foram feitas em {:.4f} segundos.".format(t)
    return f1_score(target.values, y_pred, pos_label='yes'), t


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    ltime = []
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    #print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    ltime.append(train_classifier(clf, X_train, y_train))
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    f1t, time = predict_labels(clf, X_train, y_train)
    ltime.append(time)
    #print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(f1t)
    f1tt, time = predict_labels(clf, X_test, y_test)
    ltime.append(time)
    #print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(f1tt)
    #print ""
    ltime.extend([f1t,f1tt])
    ltime.insert(0,clf.__class__.__name__)    
    ltime.insert(1,len(X_train))
    return ltime

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [50]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn import svm
import matplotlib.pyplot as plt
import matplotlib #only needed to determine Matplotlib version number
matplotlib.style.use('ggplot')

# TODO: Inicialize os três modelos
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier(random_state=0)
clf_C = svm.SVC(random_state=0)

ldata = []
for clf in [clf_A,clf_B,clf_C]:
    for size in [100,200,300]:
        ldata.append(train_predict(clf, X_train[:size], y_train[:size], X_test, y_test))

pd.set_option('max_rows', 100)
pd.set_option('expand_frame_repr', False)

cols = ['CLF','QTD','Tempo_Treinamento','T_Est_Treino','T_Est_Teste','F1_Treino','F1_Teste']
df = pd.DataFrame(data = ldata, columns=cols)
print df

df_mean = df.groupby(['CLF'],sort='F1_Teste').mean()
df_mean.columns = ['Média','Tempo_Treinamento','T_Est_Treino','T_Est_Teste','F1_Treino',\
                    'F1_Teste'] 
print df_mean

df_max = df.groupby(['CLF'],sort='F1_Teste').max()
df_max.columns = ['Máximo','Tempo_Treinamento','T_Est_Treino','T_Est_Teste','F1_Treino',\
                    'F1_Teste'] 
print df_max, '\n'

print clf_C , '\n'

                      CLF  QTD  Tempo_Treinamento  T_Est_Treino  T_Est_Teste  F1_Treino  F1_Teste
0              GaussianNB  100              0.002         0.001        0.000   0.553191  0.612245
1              GaussianNB  200              0.002         0.001        0.001   0.829630  0.766917
2              GaussianNB  300              0.001         0.001        0.000   0.792176  0.738462
3  DecisionTreeClassifier  100              0.002         0.000        0.000   1.000000  0.770370
4  DecisionTreeClassifier  200              0.003         0.001        0.000   1.000000  0.725806
5  DecisionTreeClassifier  300              0.006         0.000        0.001   1.000000  0.666667
6                     SVC  100              0.001         0.001        0.001   0.795181  0.805031
7                     SVC  200              0.004         0.002        0.001   0.805970  0.805031
8                     SVC  300              0.007         0.005        0.002   0.802395  0.805031
                    

### Resultados Tabulados
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - GaussianNB**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |  0.0000              |  0.0000                     | 0.553191                   |          0.612245          |
| 200                                |  0.0000              |  0.0000                     | 0.829630                   |         0.766917          |
| 300                                |  0.0000              |  0.0000                     | 0.792176                   |        0.738462          |

** Classificador 2 - DecisionTreeClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |   0.0000             | 0.0000                      | 1.0000                     |         0.770370           |
| 200                                |   0.0160             | 0.0000                      | 1.0000                     |         0.725806           |
| 300                                |   0.0000             | 0.0000                      | 1.0000                     |        0.666667           |

** Classificador 3 - SVC**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |  0.0000              | 0.0000                      | 0.795181                   |        0.805031      |
| 200                                |  0.0000              | 0.0000                      | 0.805970                   |          0.805031      |
| 300                                |  0.0160              | 0.0000                      | 0.802395                   |          0.805031      |


## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **
Como se pode observar pelos resultados, o melhor modelo entre os três foi o SVM (SVC no SKLEARN). Pois obteve o F1 máximo de 80% e F1 médio de também 80% durante o teste. Isso contra os 77% de F1 máximo da Árvore de decisão e 76% de F1 do GaussianNB. Mas na média, obtiveram os valores de 72% e 70% de F1, respectivamente. Em termos de desempenho, o modelo GaussianNB obteve melhores resultados no caso do treinamento e o pior, como era esperado, foi o SVM. Em relação ao testes os três modelos obtiveram ótimos desempenhos.  

De forma geral, os resultados mostram que essa configuração de execução não garante que os modelos aprendam de forma consistente. Isso pode ser explicado ou pela falta de calibração dos modelos, ou pela não utilização de validação cruzada ou pela forma como os dados se apresentam de forma que os modelos não conseguem absorver sua complexidade, ou até pela tamanho do conjunto de dados, o que faz com que os modelos tendem a dar pouco atenção aos dados (como no caso do GaussianNB) ou dar muita atenção a eles (no caso da Árvore de decisão).  

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **
O modelo SVM (Suport Vector Machine), de forma geral e simples, separa os dados de um conjunto de dados linearmente, cada parte com um rótulo. Por exemplo, uma parte pode ser rotulada com os dados considerados verdadeiros e a outra como falso, para uma classificação binária. Isso é visto na figura abaixo. Nessa figura vemos os dados distribuídos num plano e uma linha separa os dados em dois grupos, os dois pontos vermelhos e dos pontos azuis. 

![Título da imagem](./svm1.png)

Para se alcançar a melhor linha que separa de forma mais precisa os dados nos dois grupos faz-se necessário treinar o SVM e ajustá-lo. O diferencial do SVM é que durante o treinamento busca-se encontrar a melhor linha capaz de separar os dados dentre infinitas possibilidades, conforme a figura seguinte. Nesta figura, pode-se verificar que a melhor linha de corte é aquela que busca maximizar a margem de seperação (as distâncias entre as linhas b11 e b12 e entre b21 e b22). Neste caso, a linha B1 apresenta uma melhor margem de separação dos dados em relação a linha B2.

![Título da imagem](./svm3.png)

Além disso o SVM é capaz de lidar com separações dos dados de forma não linear, trabalhando com dimensoões maiores, ou seja, consegue lidar com uma grande quantidade de atributos. Nessa última figura é possivel ver a curva que separa os pontos azuis dos pontos vermelho. Essa característica é um grande diferencial do SVM em relação a outros modelos.

![Título da imagem](./svm2.png)


    

### Implementação: Calibrando o Modelo
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [49]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn import grid_search

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {    
    'kernel': ['rbf','linear'],     
    'C': [0.001,0.01,0.1, 1.0, 10.0,100.0]
} 

# TODO: Inicialize o classificador
clf = svm.SVC(random_state=0)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = grid_search.GridSearchCV(clf, parameters,scoring=f1_scorer,cv=2)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf,'\n'
print "Melhor Score =", grid_obj.best_score_
print "Melhores parâmetros = ", grid_obj.best_params_,'\n'

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetros
#print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
f1, t = predict_labels(clf, X_train, y_train)
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(f1)
f1, t = predict_labels(clf, X_test, y_test)
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(f1)

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False) 

Melhor Score = 0.818269550555
Melhores parâmetros =  {'kernel': 'linear', 'C': 10.0} 

O modelo calibrado tem F1 de 0.8291 no conjunto de treinamento.
O modelo calibrado tem F1 de 0.8105 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

O SVM calibrado obteve 82% de F1 no treinamento e 81% de F1 no teste. Os melhores parâmetros encnotrados pela busca para calibrar o SVM foram Kernel = linear e C = 10.0. Os resultados obtiveram uma leve melhora após a normalização dos dados na fase de processamento dos dados das colunas do conjunto de dados.

Comparando com o SVM não calibrado: em relaçao ao teste, houve uma melhora em torno de 1% em relação ao F1 do SVM não calibrado, tanto no valor médio quando no valor máximo (conjunto de treino de tamanho 300). Em relação ao treino, houve um aumento de 2% de F1, tanto na média quanto no valor máximo. 

Isso mostra que houve ganhos no uso de grid search para se obter os parâmetros que dão melhores resultados ao SVM. 

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.